In [3]:
#Las cosas importantes porque son importadas

import cv2  
import math 

from ultralytics import YOLO

from collections import defaultdict
import numpy as np

import cv2
import pytesseract
from pytesseract import Output

model = YOLO('yolov11-plates/weights/yolov11-plates-best.pt') #Contenedores

filename = "Parking.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/771) /home/oscar/Info/VC/Entrega/PracticasVC/VC_P4/Parking.mp4: 384x640 (no detections), 52.4ms
video 1/1 (frame 2/771) /home/oscar/Info/VC/Entrega/PracticasVC/VC_P4/Parking.mp4: 384x640 (no detections), 9.6ms
video 1/1 (frame 3/771) /home/oscar/Info/VC/Entrega/PracticasVC/VC_P4/Parking.mp4: 384x640 (no detections), 12.0ms
video 1/1 (frame 4/771) /home/oscar/Info/VC/Entrega/PracticasVC/VC_P4/Parking.mp4: 384x640 (no detections), 9.0m

In [6]:
from ultralytics import YOLO
import cv2
import math
import pytesseract


model_person_vehicle = YOLO("yolo11n.pt")

model_license_plate = YOLO("yolov11-plates/weights/yolov11-plates-last.pt")

classNames = ["person", "bicycle", "car"]

#pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'
custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'


cap = cv2.VideoCapture("C0142.MP4")
#cap = cv2.VideoCapture("capri.mp4")


# Contadores de ID para cada clase
person_counter = 0
car_counter = 0

# Diccionario para asignar IDs únicos según el objeto (usando su track_id)
tracked_ids = {}
fotograma = 0

def write_line(result):
    csv_line = ""
    for i in range(0, len(result)-2):
        csv_line += str(result[i]) + ","
    csv_line += result[len(result)-2] + result[len(result)-1]
    
    nombre_archivo = "archivo.csv"
    with open(nombre_archivo, 'a') as archivo:
        archivo.write(csv_line)


while (cap.isOpened()):
    print("entre")
    ret, frame = cap.read()
    fotograma += 1

    if not ret:
        break

    results_person_vehicle = model_person_vehicle.track(frame, persist=True, classes = [0,1,2])
    
    # Iterar sobre cada detección
    for result in results_person_vehicle:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])  # Clase de la detección
            track_id = int(box.id[0]) if box.id is not None else -1

            # Comprobar si el ID de seguimiento ya está registrado
            if track_id not in tracked_ids:
                # Asignar un nuevo ID basado en la clase del objeto
                if cls == 0:  # Persona
                    person_counter += 1
                    tracked_ids[track_id] = f"P{person_counter}"
                elif cls == 2:  # Coche
                    car_counter += 1
                    tracked_ids[track_id] = f"C{car_counter}"
                else:
                    # Si es una clase diferente, no incrementar contadores
                    tracked_ids[track_id] = f"{track_id}"

            # Obtener el prefijo con el ID específico de la clase
            prefixed_id = tracked_ids[track_id]

            x1, y1, x2, y2, mx1, my1, mx2, my2, xp1, yp1, xp2, yp2 = 0,0,0,0,0,0,0,0,0,0,0,0
            avg_confidence = 0
            license_plate_text = ""
            matricula = "No"
            if cls == 0 or cls == 2:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
            
                            # Confianza
                confidence = math.ceil((box.conf[0]*100))/100

                print("Confianza --->",confidence)

                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255 * 2:
                    R, G, B = 255, 255, escala - 255 * 2
                elif escala >= 255:
                    R, G, B = 255, escala - 255, 0
                else:
                    R, G, B = escala, 0, 0



                # Dibujar el contenedor
                cv2.rectangle(frame, (x1, y1), (x2, y2), (R, G, B), 1)
                # Ponemos el texto con la clase, el ID y la confianza
                cv2.putText(frame, f"{classNames[cls]} ID:{prefixed_id} {confidence}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (R, G, B), 2)
                
                if cls == 0:
                    result = (fotograma, classNames[cls], confidence, prefixed_id, x1, y1, x2, y2, matricula, avg_confidence, mx1, my1, mx2, my2, license_plate_text, "\n")
                    write_line(result)
                elif cls == 2:
                    # Detección de matrículas
                    results_license_plate = model_license_plate(frame[y1:y2, x1:x2])

                    if(len(results_license_plate) == 0):
                        result = (fotograma, classNames[cls], confidence, prefixed_id, x1, y1, x2, y2, matricula, avg_confidence, mx1, my1, mx2, my2, license_plate_text, "\n")
                        write_line(result)
                    for result in results_license_plate:
                        boxes = result.boxes
                        for box in boxes:
                            xp1, yp1, xp2, yp2 = map(int, box.xyxy[0])
                            
                            mx1 = xp1+x1
                            my1 = yp1+y1
                            mx2 = xp2+x1
                            my2 = yp2+y1
                            cv2.rectangle(frame, (mx1, my1), (mx2, my2), (0, 255, 0), 1)

                            if mx1 == 0 and my1 == 0 and mx2 == 0 and my2 == 0:
                                matricula = "No"
                                continue

                            # Detección de matrículas
                            license_plate = frame[my1:my2, mx1:mx2]

                            # Canny
                            gray_plate = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)
                            gaussian_plate = cv2.GaussianBlur(gray_plate, (3, 3), 0)
                            _, binary_plate = cv2.threshold(gaussian_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                            # Reescalar la imagen de la matrícula
                            scale_percent = 10
                            width = int(license_plate.shape[1] * scale_percent)
                            height = int(license_plate.shape[0] * scale_percent)
                            dim = (width, height)
                            license_plate_upscaled = cv2.resize(license_plate, dim, interpolation = cv2.INTER_LINEAR)
                            
                            #Mostrar la imagen de la matrícula con Canny
                            cv2.imshow("Canny", license_plate)

                            # Pasar la imagen procesada a pytesseract
                            license_plate_text = pytesseract.image_to_string(license_plate, config=custom_config)
                            
                            #Usar image_to_data para obtener detalles de detección y confianza
                            data = pytesseract.image_to_data(license_plate, config=custom_config, output_type=pytesseract.Output.DICT)

                            #Extraer el texto y calcular la confianza promedio
                            text = " ".join([data['text'][i] for i in range(len(data['text'])) if int(data['conf'][i]) > 0])
                            confidences = [int(data['conf'][i]) for i in range(len(data['conf'])) if int(data['conf'][i]) > 0]

                            #Calcular la confianza promedio
                            if confidences:
                                avg_confidence = sum(confidences) / len(confidences)
                            else:
                                avg_confidence = 0  # Si no hay palabras detectadas

                            print("Texto detectado:", text)
                            print("Confianza promedio:", avg_confidence)

                            print("Matrícula detectada:", license_plate_text)
                            matricula = "Si"
                            license_plate_text = license_plate_text.replace("\n", "")

                            # Dibuja el contenedor y matrícula
                            cv2.putText(frame, license_plate_text, [mx1, my1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            print("Matrícula -->", license_plate_text)
                    result = (fotograma, classNames[cls], confidence, prefixed_id, x1, y1, x2, y2, matricula, avg_confidence, mx1, my1, mx2, my2, license_plate_text, "\n")
                    write_line(result)
                    


                        
    # Mostrar el fotograma procesado (opcional)
    cv2.imshow("Deteccion de vehiculos y matriculas", frame)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

cap.release()
cv2.destroyAllWindows()

entre

0: 384x640 1 person, 14.6ms
Speed: 2.5ms preprocess, 14.6ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.9
Clase --> person
entre

0: 384x640 1 person, 12.4ms
Speed: 1.0ms preprocess, 12.4ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.9
Clase --> person
entre

0: 384x640 1 person, 18.9ms
Speed: 1.6ms preprocess, 18.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.9
Clase --> person
entre

0: 384x640 1 person, 13.3ms
Speed: 1.5ms preprocess, 13.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.9
Clase --> person
entre

0: 384x640 1 person, 12.1ms
Speed: 2.0ms preprocess, 12.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.91
Clase --> person
entre

0: 384x640 1 person, 12.6ms
Speed: 1.6ms preprocess, 12.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.91
Clase -

In [1]:
from ultralytics import YOLO
import cv2
import math
import pytesseract
import easyocr

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
reader = easyocr.Reader(['en'])

custom_config = r'--oem 3 --psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

img = cv2.imread('plate-example2.jpg')

if img is not None:
    #Convierte a RGB antes de procesar
    gray_plate = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray_plate, (3, 3), 0)

    _, thresh_plate = cv2.threshold(blur, 90, 190, cv2.THRESH_BINARY_INV)

    contornos, hierarchy = cv2.findContours(thresh_plate, 
        cv2.RETR_EXTERNAL , 
        cv2.CHAIN_APPROX_SIMPLE)
    
    for contorno in contornos:
        area = cv2.contourArea(contorno)
        if area > 10:
            cv2.drawContours(thresh_plate, [contorno], -1, (255,0,0), -1)
    
    #muestra la imagen procesada
    cv2.imshow("Imagen procesada", thresh_plate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    #Texto localizado
    teseracto = pytesseract.image_to_string(thresh_plate, config=custom_config)
    print("Texto detectado por Tesseract:", teseracto)

else:
    print('Error de imagen')

WARNING ⚠️ Python>=3.10 is required, but Python==3.9.20 is currently installed 
Texto detectado por Tesseract: S



In [7]:
# Import required packages
import cv2
import pytesseract

# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Read image from which text needs to be extracted
img = cv2.imread("plate-example1.png")

# Preprocessing the image starts

# Convert the image to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Performing OTSU threshold
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

# Specify structure shape and kernel size. 
# Kernel size increases or decreases the area 
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect 
# each word instead of a sentence.
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

# Applying dilation on the threshold image
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

# Finding contours
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, 
                                                 cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()

# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    
    # Drawing a rectangle on copied image
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]
    
    # Open the file in append mode
    file = open("recognized.txt", "a")
    
    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped)
    
    # Appending the text into file
    file.write(text)
    file.write("\n")
    
    # Close the file
    file.close()


cv2.imshow('Image', im2)
if cv2.waitKey(20) == 27:
    cv2.destroyAllWindows()
# This code is modified by Susobhan Akhuli

: 